In [1]:
%load_ext autoreload
# to make modification in .py file type %autoreload

%load_ext line_profiler
#for finding out how long code takes to run
#an example of how to get diagnostics on functions
#%lprun -f ff.generate_map_full ff.generate_map_full(384, A_BB, alpha_BB, beta_sigma, freqs, nu0)

In [2]:
#------packages-------------------
%pylab inline
import time
import healpy as hp
import matplotlib.pyplot as plt
import foreground_functions as ff
from scipy.optimize import curve_fit, minimize

import emcee
print(emcee.__version__)

import corner

Populating the interactive namespace from numpy and matplotlib
3.0rc2


In [3]:
#------parameters--------
A = 1.7e3
alpha = -3.0
beta = -3.2
gamma = -2.5
ells = np.arange(384) #corresponds to nside of 128
freqs = np.linspace(30, 300, 10)*1.e9
input_params = [A, alpha, beta, gamma]
in_model = ff.model(ells, freqs, input_params)
#create a filepath to store different power spectra
filepath = '/Users/alex/Documents/foreground_moments/power_spectra/'

In [4]:
#make some data
# start=time.time()
# data = ff.ps_data(ells, freqs, input_params)
# save(filepath + 'reference_ps.npy', data)
# print(time.time()-start)
data = load(filepath + 'reference_ps.npy')

In [5]:
#calculating the wignersum up to 768
%timeit ff.model(ells, freqs, input_params)

305 ms ± 6.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
#calculating wignersum up to 384 but still loading in the full 768 w3j table
%timeit ff.model(ells, freqs, input_params)

51.2 ms ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
#calculating wignersum up to 384 but still loading in the full 768 w3j table
%timeit ff.model(ells, freqs, input_params)

50.3 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%lprun -f ff.model ff.model(ells, freqs, input_params)

In [27]:
%lprun -f ff.auto1x1 ff.auto1x1(ells, freqs, input_params)

In [28]:
%lprun -f ff.get_wigner_sum ff.get_wigner_sum(ells, input_params)

In [41]:
%autoreload

In [29]:
#try different optimize options in the einsum

In [31]:
#no optimize
%timeit ff.get_wigner_sum(ells, input_params)

696 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
#optimize='optimal'
%timeit ff.get_wigner_sum(ells, input_params)

1.44 s ± 235 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
#optimize='greedy' or True
%timeit ff.get_wigner_sum(ells, input_params)

81.1 ms ± 9.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
#optimize=path
%timeit ff.get_wigner_sum(ells, input_params)

1.11 s ± 22.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%timeit ff.get_wigner_sum(ells, input_params)

72.9 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [48]:
%autoreload

In [52]:
#doing the wignersum with contract from opt_einsum

In [47]:
%timeit ff.get_wigner_sum(ells, input_params)

81.1 ms ± 6.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#okay, so opt_einsum does not appear to be any better than using einsum with an optimize tag.

In [53]:
%timeit ff.model(ells, freqs, input_params)

60.1 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
